In [2]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

In [3]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def data_extraction():
    pattern_state_name = r"\[edit\]"
    pattern_place_name = r"^[^\(\[]+"
    
    state_name = re.compile(pattern_state_name)
    place_name = re.compile(pattern_place_name)
    
    univeristy_towns = []
    
    with open("university_towns.txt") as file:  
        lines = file.readlines() 

    for line in lines:
        if state_name.search(line):
            curr_state = place_name.match(line).group(0)
        else:
            curr_town = place_name.match(line).group(0).strip()
            univeristy_towns.append([curr_state, curr_town])
    return univeristy_towns
    
def get_list_of_university_towns():  
    list_of_university_towns = data_extraction()
    
    df = pd.DataFrame(list_of_university_towns, columns =          ['State','RegionName'])
    
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [18]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows = 219, parse_cols=[4,6])
    df.columns= ['Quarter','GDP']
    df['diff'] = df['GDP'].diff()
    df['gdp_b'] = np.where(df['diff'] > 0, '1', '0')
    gdp_data = "".join(df['gdp_b'].values)
    ind = gdp_data.index("0011")
    index_start_of_recession = gdp_data.rindex("1", 0, ind) + 1
    return df.iloc[index_start_of_recession].Quarter
get_recession_start()

TypeError: read_excel() got an unexpected keyword argument 'parse_cols'

In [19]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows = 219, parse_cols=[4,6])
    df.columns= ['Quarter','GDP']
    df['diff'] = df['GDP'].diff()
    df['gdp_b'] = np.where(df['diff'] > 0, '1', '0')
    gdp_data = "".join(df['gdp_b'].values)
    ind = gdp_data.index("0011")
    index_end_of_recession = ind + len("0011") - 1
    return df.iloc[index_end_of_recession].Quarter
get_recession_end()

TypeError: read_excel() got an unexpected keyword argument 'parse_cols'

In [20]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows = 219, parse_cols=[4,6])
    df.columns= ['Quarter','GDP']
    start_index = df[df['Quarter'] == get_recession_start()].index.tolist()[0]
    end_index = df[df['Quarter'] == get_recession_end()].index.tolist()[0]
    lowest_gdp_in_recession = df['GDP'].loc[start_index : end_index + 1].min()
    lowest_gdp_in_recession_index = df[df['GDP'] == lowest_gdp_in_recession].index.tolist()[0]
    return df.iloc[lowest_gdp_in_recession_index].Quarter

get_recession_bottom()

TypeError: read_excel() got an unexpected keyword argument 'parse_cols'

In [21]:
def convert_housing_data_to_quarters():
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_data['State'] = housing_data['State'].map(states)
    cols = list(housing_data.columns)
    mdf = housing_data[cols[cols.index('2000-01'):]].rename(columns=pd.to_datetime).resample('Q',axis=1).mean().rename(columns=lambda x: '{:}q{:}'.format(x.year,x.quarter))
    mdf['RegionName'] = housing_data['RegionName']
    mdf['State'] = housing_data['State']
    mdf.set_index(['State','RegionName'], inplace = True)
    return mdf

convert_housing_data_to_quarters()

FileNotFoundError: [Errno 2] No such file or directory: 'City_Zhvi_AllHomes.csv'

In [22]:
def run_ttest():
    hdf = convert_housing_data_to_quarters()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()
    
    qrt_bfr_rec_start = hdf.columns.tolist()[hdf.columns.tolist().index(rec_start) - 1]
    hdf['PriceRatio'] = hdf[qrt_bfr_rec_start].div(hdf[rec_bottom])
    
    tuple_list = [tuple(x) for x in ul.values]
    university_towns = hdf.loc[tuple_list]
    non_university_towns = hdf.loc[~hdf.index.isin(tuple_list)]
    
    ttest_result = ttest_ind(non_university_towns['PriceRatio'], university_towns['PriceRatio'],nan_policy='omit')
    
    different = ttest_result.pvalue <0.01
    p = ttest_result.pvalue 
    
    if university_towns['PriceRatio'].mean() < non_university_towns['PriceRatio'].mean():
        better = "university town"
    else:
        better ="non-university town"
    
    return (different, p, better)   

run_ttest()

FileNotFoundError: [Errno 2] No such file or directory: 'City_Zhvi_AllHomes.csv'